<a href="https://colab.research.google.com/github/RyanDanWhite/RyanDanWhite.github.io/blob/master/Finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# detect environment and install libraries
    import google.colab  # Check if in Colab
    !pip install yfinance --upgrade --no-cache-dir
    !pip install pandas
    !pip install numpy
    !pip install requests_cache
except ImportError:
    # Assume Anaconda environment
    !conda install -c conda-forge yfinance -y
    !conda install pandas numpy requests_cache -y

Yahoo Finance API allows you to make 2000 requests per IP per hour => 48k requests per day.
https://ranaroussi.github.io/yfinance/reference/index.html

In [ ]:
#import libraries
import yfinance as yf
import pandas as pd
import numpy as np
import requests_cache
import os

In [ ]:
# start a PHP session for improved performance
# if using in anaconda this step may cause errors
cache_dir = os.path.expanduser('~/.cache/yfinance')  # Portable cache directory
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
session = requests_cache.CachedSession(cache_dir)
session.expire_after = 86400

In [ ]:
# Define the stock tickers
# Get Russell 2000 tickers
r2000 = yf.Ticker("^RUT")
r2000_tickers = r2000.tickers

# Get S&P 500 tickers
sp500 = yf.Ticker("^GSPC")
sp500_tickers = sp500.tickers

# Combine the lists
tickers = r2000_tickers + sp500_tickers

In [ ]:
# Function to calculate moving averages
def calculate_moving_averages(data, window):
    return data.rolling(window=window).mean()

In [ ]:
# Function to calculate RSI
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [ ]:
# Function to calculate MACD
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data.ewm(span=short_window, adjust=False).mean()
    long_ema = data.ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal

In [ ]:
# Download data and calculate indicators
for ticker in tickers:
    try:
        data = yf.download(ticker, period="1y")
        if 'Close' in data.columns:
            data['SMA_50'] = calculate_moving_averages(data['Close'], 50)
            data['SMA_200'] = calculate_moving_averages(data['Close'], 200)
            data['MACD'], data['Signal_Line'] = calculate_macd(data['Close'])
            data['RSI'] = calculate_rsi(data['Close'])
        else:
            print(f"Warning: 'Close' column not found for {ticker}")
    except Exception as e:
        print(f"Error processing {ticker}: {e}")


In [ ]:
 # Save to CSV
    data.to_csv(f"{ticker}_data.csv")